In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os

# Add project root to Python path
notebook_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in globals() else os.getcwd()
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from libs import pnllib
from tools.posanalysis import main
import pandas as pd
import numpy as np

pnllib.set_log_level(pnllib.ERROR)

In [ ]:
dct=main()

In [ ]:
d=dct["11/21/2025"]
d["gap"]=-d["Cost Basis"] - d.PnL
d.loc[d.PnL + d["Cost Basis"] < -25, ["Symbol", "Quantity", "Cost Basis", "PnL", "gap", "Margin"] ].reset_index()

In [ ]:
Total margin 117241
Weekly DataFrame:
            Cost Basis  Quantity  Premium   PnL  Margin
Date                                                   
2025-11-14        1580        28     1581  1197   10545
2025-11-21        7420       111     7421  3654   24961
2025-11-28        2229        38     2230  1350   11518
2025-12-05        2881        49     2882   192   15560
2025-12-12        2894        44     2895  -220   13875
2025-12-19        5371        89     5371  -945   29778
2025-12-26         306         5      307   -85    2047


Monthly Aggregated DataFrame:
         Cost Basis  Quantity  Premium   PnL  Margin
Month                                               
2025-11       11229       177    11232  6201   47024
2025-12       11452       187    11455 -1058   61260

3588

